In [1]:
import pandas as pd

red_url = 'https://github.com/PinkWink/ML_tutorial/raw/master/dataset/winequality-red.csv'
white_url = 'https://github.com/PinkWink/ML_tutorial/raw/master/dataset/winequality-white.csv'

red_wine = pd.read_csv(red_url, sep=';')
white_wine = pd.read_csv(white_url, sep=';')

red_wine['color'] = 1
white_wine['color'] = 0

wine = pd.concat([red_wine, white_wine])

In [2]:
wine['taste'] = [1. if grade > 5 else 0. for grade in wine['quality']]

X = wine.drop(['taste', 'quality'], axis=1)
y = wine['taste']

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

params = {'max_depth': [2, 4, 7, 10]}
wine_tree = DecisionTreeClassifier(max_depth=2, random_state=13)

gridsearch = GridSearchCV(estimator=wine_tree, param_grid=params, cv=5)
gridsearch.fit(X, y)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=2, random_state=13),
             param_grid={'max_depth': [2, 4, 7, 10]})

In [4]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(gridsearch.cv_results_)

{   'mean_fit_time': array([0.01417398, 0.01675839, 0.03031659, 0.04707532]),
    'mean_score_time': array([0.00258899, 0.00239038, 0.00258579, 0.00199308]),
    'mean_test_score': array([0.6888005 , 0.66356523, 0.65340854, 0.64401587]),
    'param_max_depth': masked_array(data=[2, 4, 7, 10],
             mask=[False, False, False, False],
       fill_value='?',
            dtype=object),
    'params': [   {'max_depth': 2},
                  {'max_depth': 4},
                  {'max_depth': 7},
                  {'max_depth': 10}],
    'rank_test_score': array([1, 2, 3, 4]),
    'split0_test_score': array([0.55230769, 0.51230769, 0.50846154, 0.51615385]),
    'split1_test_score': array([0.68846154, 0.63153846, 0.60307692, 0.60076923]),
    'split2_test_score': array([0.71439569, 0.72363356, 0.68360277, 0.66743649]),
    'split3_test_score': array([0.73210162, 0.73210162, 0.73672055, 0.71054657]),
    'split4_test_score': array([0.75673595, 0.7182448 , 0.73518091, 0.72517321]),
    'std

In [5]:
gridsearch.best_estimator_

DecisionTreeClassifier(max_depth=2, random_state=13)

In [6]:
gridsearch.best_score_

0.6888004974240539

In [7]:
gridsearch.best_params_

{'max_depth': 2}

## pipeline에 GridSearch 적용

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

estimators = [
    ('scaler', StandardScaler()),
    ('clf', DecisionTreeClassifier(random_state=13))
]

pipe = Pipeline(estimators)

In [9]:
param_grid = {'clf__max_depth': [2, 4, 7, 10]}

gridsearch_pipe = GridSearchCV(estimator=pipe, param_grid=param_grid, cv=5)
gridsearch_pipe.fit(X, y)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('clf',
                                        DecisionTreeClassifier(random_state=13))]),
             param_grid={'clf__max_depth': [2, 4, 7, 10]})

In [10]:
gridsearch_pipe.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('clf', DecisionTreeClassifier(max_depth=2, random_state=13))])

In [11]:
gridsearch_pipe.best_score_

0.6888004974240539

In [12]:
gridsearch_pipe.best_params_

{'clf__max_depth': 2}

In [13]:
score_df = pd.DataFrame(gridsearch_pipe.cv_results_)
score_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014572,0.003720,0.003588,0.000492,2,{'clf__max_depth': 2},0.552308,0.688462,0.714396,0.732102,0.756736,0.688800,0.071799,1
1,0.019156,0.000746,0.002395,0.000490,4,{'clf__max_depth': 4},0.512308,0.631538,0.723634,0.732102,0.718245,0.663565,0.083905,2
2,0.032905,0.001664,0.002800,0.000407,7,{'clf__max_depth': 7},0.508462,0.604615,0.682063,0.736721,0.735181,0.653408,0.086993,3
3,0.051070,0.005490,0.002988,0.000624,10,{'clf__max_depth': 10},0.516154,0.602308,0.665897,0.710547,0.725173,0.644016,0.076915,4


In [14]:
score_df[['params', 'rank_test_score', 'mean_test_score', 'std_test_score']]

,params,rank_test_score,mean_test_score,std_test_score
0,{'clf__max_depth': 2},1,0.688800,0.071799
1,{'clf__max_depth': 4},2,0.663565,0.083905
2,{'clf__max_depth': 7},3,0.653408,0.086993
3,{'clf__max_depth': 10},4,0.644016,0.076915
